# Introduction

Let's investigate how we can use a Langchain SQL Agent to explore ChEMBL database.  

## What is ChEMBL?

Excerpt from https://www.ebi.ac.uk/chembl/ site: ChEMBL is a manually curated database of bioactive molecules with drug-like properties. It brings together chemical, bioactivity and genomic data to aid the translation of genomic information into effective new drugs. 



# Packages installation and import


In [1]:
!pip install -q -U langchain
!pip install -q -U openai

In [2]:
from langchain.agents import create_sql_agent 
from langchain.agents.agent_toolkits import SQLDatabaseToolkit 
from langchain.sql_database import SQLDatabase 
from langchain.llms.openai import OpenAI 
from langchain.agents import AgentExecutor 
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

# Setup the database connection

In [3]:
sqlite_path = "/kaggle/input/chembl-database/chembl_33.db"
sqlite_uri = f"sqlite:///{sqlite_path}"
db = SQLDatabase.from_uri(sqlite_uri)

# Configure ChatOpenAI   

We initialize the LLM that will be used to power the SQLAgent. For this experiment, we will use ChatGPT (gpt-3.5-turbo).

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
llm = ChatOpenAI(temperature=0, 
                 openai_api_key=user_secrets.get_secret("OPENAI_API_KEY"), 
                 model_name='gpt-3.5-turbo')

# Prepare the Langchain SQLAgent  

The Agent executor is initialized with `create_sql_agent` from Langchain, setting the LLM parameter, the toolkit parameter, and the agent type. We want to be able to monitor the entire process, therefore we set the parameter `verbose` to `True`.

In [5]:
toolkit = SQLDatabaseToolkit(db=db, 
                             llm=llm)

In [6]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

# Test the agent  


Now we will test the agent with few simple tasks.

In [7]:
question = "How many activities are?"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: action_type, activities, activity_properties, activity_smid, activity_stds_lookup, activity_supp, activity_supp_map, assay_class_map, assay_classification, assay_parameters, assay_type, assays, atc_classification, binding_sites, bio_component_sequences, bioassay_ontology, biotherapeutic_components, biotherapeutics, cell_dictionary, chembl_id_lookup, chembl_release, component_class, component_domains, component_go, component_sequences, component_synonyms, compound_properties, compound_records, compound_structural_alerts, compound_structures, confidence_score_lookup, curation_lookup, data_validity_lookup, defined_daily_dose, docs, domains, drug_indication, drug_mechanism, drug_warning, formulations, frac_classification, go_classification, hrac_classification, indication_refs, irac_classification, ligand_eff, mechanism_refs, metabolism, metabolism_refs, molecule_atc_classification, molecule_di

'There are 20,334,684 activities.'

In [8]:
question = "How many SMILES codes are?"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: action_type, activities, activity_properties, activity_smid, activity_stds_lookup, activity_supp, activity_supp_map, assay_class_map, assay_classification, assay_parameters, assay_type, assays, atc_classification, binding_sites, bio_component_sequences, bioassay_ontology, biotherapeutic_components, biotherapeutics, cell_dictionary, chembl_id_lookup, chembl_release, component_class, component_domains, component_go, component_sequences, component_synonyms, compound_properties, compound_records, compound_structural_alerts, compound_structures, confidence_score_lookup, curation_lookup, data_validity_lookup, defined_daily_dose, docs, domains, drug_indication, drug_mechanism, drug_warning, formulations, frac_classification, go_classification, hrac_classification, indication_refs, irac_classification, ligand_eff, mechanism_refs, metabolism, metabolism_refs, molecule_atc_classification, molecule_di

'2,399,743'

In [9]:
question = "List 3 molecules actives for Parkinson"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: action_type, activities, activity_properties, activity_smid, activity_stds_lookup, activity_supp, activity_supp_map, assay_class_map, assay_classification, assay_parameters, assay_type, assays, atc_classification, binding_sites, bio_component_sequences, bioassay_ontology, biotherapeutic_components, biotherapeutics, cell_dictionary, chembl_id_lookup, chembl_release, component_class, component_domains, component_go, component_sequences, component_synonyms, compound_properties, compound_records, compound_structural_alerts, compound_structures, confidence_score_lookup, curation_lookup, data_validity_lookup, defined_daily_dose, docs, domains, drug_indication, drug_mechanism, drug_warning, formulations, frac_classification, go_classification, hrac_classification, indication_refs, irac_classification, ligand_eff, mechanism_refs, metabolism, metabolism_refs, molecule_atc_classification, molecule_di

'The 3 molecules that are active for Parkinson are ROPINIROLE with standard values of 7.2, 19, and 1706.'

In [10]:
question = "For what disease is active ROPINIROLE?"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: action_type, activities, activity_properties, activity_smid, activity_stds_lookup, activity_supp, activity_supp_map, assay_class_map, assay_classification, assay_parameters, assay_type, assays, atc_classification, binding_sites, bio_component_sequences, bioassay_ontology, biotherapeutic_components, biotherapeutics, cell_dictionary, chembl_id_lookup, chembl_release, component_class, component_domains, component_go, component_sequences, component_synonyms, compound_properties, compound_records, compound_structural_alerts, compound_structures, confidence_score_lookup, curation_lookup, data_validity_lookup, defined_daily_dose, docs, domains, drug_indication, drug_mechanism, drug_warning, formulations, frac_classification, go_classification, hrac_classification, indication_refs, irac_classification, ligand_eff, mechanism_refs, metabolism, metabolism_refs, molecule_atc_classification, molecule_di

'Restless Legs Syndrome, Stroke, Cocaine-Related Disorders, Fibromyalgia, Hyperprolactinemia, Parkinson Disease, and Unverricht-Lundborg Syndrome.'